In [1]:
import os
import sys
import zipfile

In [2]:
! gdown https://drive.google.com/u/0/uc?id=1SPsS1O8kvE_prMLeq3DtbDG7WvZ_7SyZ&export=download

Downloading...
From: https://drive.google.com/u/0/uc?id=1SPsS1O8kvE_prMLeq3DtbDG7WvZ_7SyZ
To: /content/[원천]세종특별자치시_001.zip
100% 5.56G/5.56G [00:59<00:00, 93.3MB/s]


In [3]:
!ls

'[원천]세종특별자치시_001.zip'	 sample_data


In [4]:
# zip 파일 읽어보기
path = '/content/[원천]세종특별자치시_001.zip'
file1 = zipfile.ZipFile(path, 'r')
with file1 :
  name_list = file1.namelist()
  print(len(name_list)) # 1588개의 이미지 명
  print(name_list[:5])

1588
['6.25격전지개미고개/', '6.25격전지개미고개/6.25격전지개미고개_001_40613647.JPG', '6.25격전지개미고개/6.25격전지개미고개_003_40613648.JPG', '6.25격전지개미고개/6.25격전지개미고개_004_40613649.JPG', '6.25격전지개미고개/6.25격전지개미고개_005_40613650.JPG']


In [5]:
# 파일 명 = 랜드마크 명이므로 split 해서 label 리스트로 바인딩
labels = []
for name in name_list :
  labels.append(name.split('/')[0])

print(labels[:3], len(labels))

['6.25격전지개미고개', '6.25격전지개미고개', '6.25격전지개미고개'] 1588


In [6]:
# 중복되는 label 제외한 분류할 클래스 수 확인
len(set(labels))
# set(labels)내에 값들 확인 결과 새뜰1단지 아파트, 새뜰2단지 아파트와 같은 label들 존재.

84

In [7]:
path = '/content/[원천]세종특별자치시_001.zip'
file1 = zipfile.ZipFile(path, 'r')
file1.extractall('/content/[원천]세종특별자치시_001')
file1.close()

In [8]:
# train/test 폴더 생성
path2 = '/content/[원천]세종특별자치시_001'
os.mkdir('Landmark')

for t in ['train','valid', 'test'] :
  dir_path = f'Landmark/{t}'
  os.mkdir(dir_path)

In [9]:
import shutil
# train, test 셋 분리
train_path = []
valid_path = []
test_path = []

# train
for img_path,_,img_name in os.walk(path2) :  # os.walk(path) : 파일의 경로, [], 파일 명 리턴
  if len(img_name) > 0 :
    n = int(len(img_name)*0.2)     # 각 label에 들어있는 이미지 수의 20%를 테스트 데이터로 사용
    name = img_name[n:-1] # 이미지 파일명
    label = img_path.split('/')[-1]
    os.mkdir(f'Landmark/train/{label}')
    for a in name :
      total_path = f'{img_path}/{a}'
      shutil.move(total_path, f'Landmark/train/{label}')  # shutil.move를 이용해 기존 경로에서 train에 각 label로 경로 변경

# valid
for img_path,_,img_name in os.walk(path2) :  # os.walk(path) : 파일의 경로, [], 파일 명 리턴
  if len(img_name) > 0 : # os.walk(path2)의 첫번째 값은 [] 이므로 그 이후 값부터 저장
    name = img_name[-1] # 이미지 파일명
    label = img_path.split('/')[-1]
    os.mkdir(f'Landmark/valid/{label}')
    total_path = f'{img_path}/{name}'
    shutil.move(total_path, f'Landmark/valid/{label}') # shutil.move를 이용해 기존 경로에서 valid에 각 label로 경로 변경

# test
for img_path,_,img_name in os.walk(path2) :
  if len(img_name) > 0 :
    # 이미 shutil.move로 파일 이동 했기 때문에 나머지 남은 파일들은 바로 test로 저장
    name= img_name
    label = img_path.split('/')[-1]
    os.mkdir(f'Landmark/test/{label}')
    for a in name :
      total_path = f'{img_path}/{a}'
      shutil.move(total_path, f'Landmark/test/{label}') # shutil.move를 이용해 기존 경로에서 test에 각 label로 경로 변경

In [137]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 이미지 증강
aug_gen = ImageDataGenerator(rescale=1.0/255.0,
                             rotation_range = 40,
                             width_shift_range = 0.2,
                             shear_range = 0.2,
                             zoom_range = 0.2,
                             horizontal_flip = True,
                             fill_mode = 'nearest',
                             brightness_range = [0.7,1.3])

train = aug_gen.flow_from_directory('/content/Landmark/train',
                                        target_size = (150,150),
                                        batch_size = 20,
                                        class_mode = 'categorical', # 이진분류
                                          )
valid = ImageDataGenerator(rescale = 1.0/255.0).flow_from_directory('/content/Landmark/valid',
                                                                      target_size = (150,150),
                                                                      batch_size = 20,
                                                                      class_mode = 'categorical', # 이진분류
                                                                      )

Found 1165 images belonging to 84 classes.
Found 84 images belonging to 84 classes.


In [186]:
import matplotlib.pyplot as plt
from tensorflow.keras import *
import tensorflow as tf

# base model 사용 X
def build_model() :
  x = layers.Input(shape=(150,150,3))
  x2 = x
  y = layers.Conv2D(filters = 64,kernel_size = 7, strides = 2,  padding = 'SAME', activation = 'relu')(x)
  y = layers.MaxPool2D(pool_size=3,strides = 2)(y)
  x1
  y = layers.Conv2D(filters = 128,kernel_size = 3, strides = 3,  padding = 'SAME', activation = 'relu')(y)
  y = layers.MaxPool2D()(y)
  y = layers.Conv2D(filters = 256,kernel_size = 3, strides = 3,  padding = 'SAME', activation = 'relu')(y)
  y = layers.GlobalAveragePooling2D()(y)
  y = layers.Dense(84, activation = 'softmax')(y)
  return models.Model(x,y, name = 'First_model')

In [187]:
model = build_model()
model.summary()

Model: "First_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_55 (InputLayer)       [(None, 150, 150, 3)]     0         
                                                                 
 conv2d_149 (Conv2D)         (None, 75, 75, 64)        9472      
                                                                 
 max_pooling2d_103 (MaxPooli  (None, 37, 37, 64)       0         
 ng2D)                                                           
                                                                 
 conv2d_150 (Conv2D)         (None, 13, 13, 128)       73856     
                                                                 
 max_pooling2d_104 (MaxPooli  (None, 6, 6, 128)        0         
 ng2D)                                                           
                                                                 
 conv2d_151 (Conv2D)         (None, 2, 2, 256)         

In [183]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['acc'] )
                       
history = model.fit(train,
                    epochs = 20,
                    validation_data = valid,
                    ) 

Epoch 1/20
34/59 [================>.............] - ETA: 2:00 - loss: 4.4346 - acc: 0.0103

KeyboardInterrupt: ignored